# [SK 07 - AI Foundry Agents with Semantic Kernel vs. AI Foundry SDK's](https://github.com/microsoft/semantic-kernel/tree/main/python/samples/getting_started_with_agents/azure_ai_agent)
- How to use Azure AI Agents with Semantic Kernel.<br/>
Dependencies: `pip install semantic-kernel[azure]`. [Sample](https://github.com/microsoft/semantic-kernel/blob/main/python/samples/getting_started_with_agents/azure_ai_agent/step1_azure_ai_agent.py)<br/><br/>

Note: it's worth to review the usage of AI Foundry Agents
- with [Python AI Foundry SDK](https://github.com/maurominella/aaas)
- with [C#](https://github.com/maurominella/aaas/tree/main/FoundryAgents06%20-%20AI%20Foundry%20Agent%20with%20BingGroundingTool)

# Constants and Libraries

In [ ]:
import os
from dotenv import load_dotenv # requires python-dotenv
from azure.identity import DefaultAzureCredential

load_dotenv("./../config/credentials_my.env")

agent_name                  = "aiagent-PYTHON"
instructions                = "you are a clever agent"
user_inputs = [
    "Toggle the status of my second light.",
]

plugin_name                 = "Lights"

project_connection_string   = os.environ["PROJECT_CONNECTION_STRING"]
model_deployment_name       = os.environ['AZURE_OPENAI_CHAT_DEPLOYMENT_NAME']

credential                  = DefaultAzureCredential()

# Native Plugin

In [ ]:
# First, we define the plugin through its class...

class LightsPlugin:
    from typing import Annotated
    from semantic_kernel.functions import kernel_function
    
    lights = [
        {"id": 0, "name": "Table Lamp", "is_on": False},
        {"id": 1, "name": "Porch light", "is_on": False},
        {"id": 2, "name": "Chandelier", "is_on": True},
    ]

    @kernel_function(
        name="get_lights", # <<<=== DIFFERENT FROM THE FUNCTION NAME <get_state>, which will be ignored
        description="Gets a list of lights and their current state",
    )
    def get_state(
        self,
    ) -> Annotated[str, "the output is a string"]:
        """Gets a list of lights and their current state."""
        return self.lights

    @kernel_function(
        name="change_state",
        description="Changes the state of the light",
    )
    def change_state(
        self,
        id: int,
        is_on: bool,
    ) -> Annotated[str, "the output is a string"]:
        """Changes the state of the light."""
        for light in self.lights:
            if light["id"] == id:
                light["is_on"] = is_on
                return light
        return None

# AI FOUNDRY PROJECT CLIENT

## AI Foundry SDK

In [ ]:
from azure.ai.projects import AIProjectClient

aifoundry_project_client = AIProjectClient.from_connection_string(
    credential=credential, 
    conn_str=project_connection_string,
)

aifoundry_project_client

## Semantic Kernel SDK

In [ ]:
from semantic_kernel.agents.azure_ai import AzureAIAgent

sk_project_client = AzureAIAgent.create_client(
    credential=credential,
    conn_str=project_connection_string,
)

sk_project_client

# AI FOUNDRY AGENT

## AI Foundry SDK
Single step:
- `create_agent` for agent **creation**

In [ ]:
aifoundry_ai_agent = aifoundry_project_client.agents.create_agent(
    model=model_deployment_name,
    name=f"{agent_name}_aifoundry",
    instructions=instructions
)

aifoundry_ai_agent

## Semantic Kernel SDK
Two steps:
- `create_agent` for **agent definition**
- `AzureAIAgent` for **client creation** (including **kernel**)

In [ ]:
sk_ai_agent_definition = await sk_project_client.agents.create_agent(
    model=model_deployment_name,
    name=f"{agent_name}_SK",
    instructions=instructions
)
sk_ai_agent_definition

In [ ]:
sk_ai_agent = AzureAIAgent(
    client=sk_project_client,
    definition=sk_ai_agent_definition,
)
sk_ai_agent

# ADD PLUGIN TO THE KERNEL

## AI Foundry SDK

In [ ]:
from azure.ai.projects.models import FunctionTool, ToolSet

## Semantic Kernel SDK

In [ ]:
sk_ai_agent.kernel.add_plugin(
    plugin=LightsPlugin(),
    plugin_name=plugin_name,
)
sk_ai_agent

# CREATE A NEW THREAD

## AI Foundry SDK

In [ ]:
aifoundry_thread = aifoundry_project_client.agents.create_thread()
aifoundry_thread

## Semantic Kernel SDK

In [ ]:
sk_thread = await sk_project_client.agents.create_thread()
sk_thread

# MESSAGE(S) CREATION AND INVOKATION

## AI Foundry SDK

In [ ]:
for user_input in user_inputs:
    message = aifoundry_project_client.agents.create_message(
        thread_id=aifoundry_thread.id, 
        role="user", 
        content=user_input,
    )

## Semantic Kernel SDK

In [ ]:
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

for user_input in user_inputs:
    await sk_ai_agent.add_chat_message(
        thread_id=sk_thread.id,
        message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
    )

# RUN THE AGENT

## AI Foundry SDK

In [ ]:
run = aifoundry_project_client.agents.create_and_process_run(
    thread_id=aifoundry_thread.id, 
    assistant_id=aifoundry_ai_agent.id)

print(f"Run finished with status: {run.status}. Run: {run}")

In [ ]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = aifoundry_project_client.agents.list_messages(thread_id=aifoundry_thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

## Semantic Kernel SDK

In [ ]:
async for content in sk_ai_agent.invoke(        
    thread_id=sk_thread.id,
    temperature=0.2,  # override the agent-level temperature setting with a run-time value
):
    print(f"# Agent: {content.to_dict()}")

# HIC SUNT LEONES

# TEARDOWN

In [ ]:
from openai import AzureOpenAI

# Create the client
client = AzureOpenAI(
    # api_key        = os.getenv("AZURE_OPENAI_API_KEY"),  
    # api_version    = os.getenv("AZURE_OPENAI_API_VERSION"), # at least 2024-02-15-preview
    # azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)
print(f"client.base_url: {client.base_url}")

In [ ]:
# delete files

files_list = client.files.list().data
print(f"There are {len(files_list)} files to delete")

i = 0
for file in files_list:
    i += 1
    print(f"File {i}/{len(files_list)}: {file.filename} (id={file.id}) is being deleted...")
    client.files.delete(file.id) # un-comment this line if you want to delete it

In [ ]:
print(f"Deleting thread {thread_id}...")
#client.beta.threads.delete(thread.id)

In [ ]:
# delete all assistants

assistants_list = client.beta.assistants.list().data
print(f"There are {len(assistants_list)} assistants to delete")

i = 0
for assistant in assistants_list:
    i += 1
    print(f"Assistant {i}/{len(assistants_list)}: Assistant {assistant.name} ({assistant.id})) is being deleted...")
    client.beta.assistants.delete(assistant_id=assistant.id) # un-comment this line if you want to delete it

# Check the message history through the thread id
**IMPORTANT**
- The Assistant Agent automatically manages the history through the thread.
- The list of message history in the thread starts with the most recent one to the oldest one.

In [ ]:
i =0
async for message in agent.get_thread_messages(thread_id):
    i += 1
    print(f">>> Message {i} - {message}\n")

# Additional tests. Run multiple times to toggle the first light.

In [ ]:
message = ChatMessageContent(role=AuthorRole.USER, content="Toggle the first light and give me the status of all my lights.")

await agent.add_chat_message(thread_id=thread_id, message=message)

async for message in agent.invoke(thread_id=thread_id):
    print(message)

messages = [message async for message in agent.get_thread_messages(thread_id)]
print(f"\n\nHere are all the {len(messages)} messages:\n")
for i, message in enumerate(reversed(messages), 1):
    print(f">>> Message {i} - {message}\n")